In [40]:
import os
import pathlib
import cv2 
import numpy as np
import keras
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import keras.utils as utils
from keras.layers import Conv2D, MaxPool2D
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from sklearn.utils import shuffle


In [41]:
#!unzip data.zip

In [42]:
# Read the 200 images
def read_images(path):
    letters = {"A", "B", "C", "D", "E"}
    Images = []
    label = []
    n=0

    for i in letters:
        __dir = '/content/data/person{}/{}/'.format(i, path)

        images_path = os.listdir(__dir)
        for image_path in images_path:
            if pathlib.Path(image_path).suffix[1] == 'p':
                full_path = os.path.join(__dir, image_path)
                image = cv2.imread(full_path)
                image =cv2.resize(image,(256,256))
                Images.append(image)
                label.append(n)
        
        n = n+1
    
    return np.array(Images)/255.0,np.array(label)

In [44]:
train_images,train_labels = read_images("Train")
test_images,test_labels = read_images("Test")
train_images, train_labels = shuffle(train_images, train_labels, random_state=0)

In [45]:
print ("X_train shape: " + str(train_images.shape))
print ("Y_train shape: " + str(train_labels.shape))
print ("X_test shape: " + str(test_images.shape))
print ("Y_test shape: " + str(test_labels.shape))

X_train shape: (200, 256, 256, 3)
Y_train shape: (200,)
X_test shape: (40, 256, 256, 3)
Y_test shape: (40,)


In [46]:
model = models.Sequential()
model.add(layers.Conv2D(filters = 8 , kernel_size= (5,5), strides = (1,1), padding='same', activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D(pool_size=(8,8), strides=(8, 8), padding='same'))
model.add(layers.Conv2D(filters = 16 , kernel_size= (3,3), strides = (1,1), padding='same', activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(4,4), strides=(4, 4), padding='same'))

In [31]:
model.add(layers.Flatten())
model.add(layers.Dense(5))
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 256, 256, 8)       608       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 32, 32, 8)        0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 32, 32, 16)        1168      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 8, 8, 16)         0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 5)                

In [32]:
model.compile(optimizer='adam', 
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=7,
                                            restore_best_weights=True)
history = model.fit(train_images,train_labels, epochs=30,callbacks=callback,validation_split=.2)

Epoch 1/30
5/5 [==============================] - 7s 1s/step - loss: 1.6800 - accuracy: 0.1750 - val_loss: 1.6136 - val_accuracy: 0.1500
Epoch 2/30
5/5 [==============================] - 6s 1s/step - loss: 1.5114 - accuracy: 0.3125 - val_loss: 1.4750 - val_accuracy: 0.6000
Epoch 3/30
5/5 [==============================] - 6s 1s/step - loss: 1.3621 - accuracy: 0.5312 - val_loss: 1.3226 - val_accuracy: 0.5250
Epoch 4/30
5/5 [==============================] - 6s 1s/step - loss: 1.1946 - accuracy: 0.6313 - val_loss: 1.1317 - val_accuracy: 0.6250
Epoch 5/30
5/5 [==============================] - 7s 1s/step - loss: 0.9886 - accuracy: 0.8313 - val_loss: 0.9352 - val_accuracy: 0.8250
Epoch 6/30
5/5 [==============================] - 6s 1s/step - loss: 0.7692 - accuracy: 0.8938 - val_loss: 0.7560 - val_accuracy: 0.8250
Epoch 7/30
5/5 [==============================] - 6s 1s/step - loss: 0.5747 - accuracy: 0.9250 - val_loss: 0.5615 - val_accuracy: 0.9000
Epoch 8/30
5/5 [=========================

In [33]:
model.evaluate(test_images, test_labels)

2/2 [==============================] - 1s 122ms/step - loss: 0.0383 - accuracy: 1.0000


[0.0382622666656971, 1.0]

In [27]:
model.predict(test_images).argmax(axis = 1)

2/2 [==============================] - 0s 81ms/step


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4])

In [ ]:
!mkdir -p saved_model
model.save('saved_model/my_model') 

In [29]:
model.save("model1.hdf5")